# Sitzung 4 – Vergleich SLM vs. LLM

Vergleichen Sie die Latenz und die Qualität der Beispielantworten zwischen einem kleinen Sprachmodell und einem größeren Modell, das über Foundry Local läuft.


## ⚡ Schnellstart

**Speicheroptimierte Einrichtung (Aktualisiert):**  
1. Modelle wählen automatisch CPU-Varianten aus (funktioniert auf jeder Hardware)  
2. Verwendet `qwen2.5-3b` anstelle von 7B (spart ~4GB RAM)  
3. Automatische Port-Erkennung (keine manuelle Konfiguration erforderlich)  
4. Benötigter Gesamtspeicher: ~8GB empfohlen (Modelle + Betriebssystem)  

**Terminal-Einrichtung (30 Sekunden):**  
```bash
foundry service start
foundry model run phi-4-mini
foundry model run qwen2.5-3b
```
  
Dann führen Sie dieses Notebook aus! 🚀  


### Erklärung: Abhängigkeitsinstallation
Installiert die minimal erforderlichen Pakete (`foundry-local-sdk`, `openai`, `numpy`) für Timing- und Chat-Anfragen. Kann gefahrlos mehrfach ausgeführt werden, da es idempotent ist.


# Szenario
Vergleichen Sie ein repräsentatives kleines Sprachmodell (SLM) mit einem größeren Modell anhand eines einzelnen Prompts, um die Kompromisse zu veranschaulichen:
- **Latenzunterschied** (Zeit in Sekunden)
- **Token-Nutzung** (falls verfügbar) als Proxy für den Durchsatz
- **Qualitative Beispielausgabe** für eine schnelle Einschätzung
- **Geschwindigkeitsberechnung**, um Leistungsgewinne zu quantifizieren

**Umgebungsvariablen:**
- `SLM_ALIAS` - Kleines Sprachmodell (Standard: phi-4-mini, ~4GB RAM)
- `LLM_ALIAS` - Größeres Sprachmodell (Standard: qwen2.5-7b, ~7GB RAM)
- `COMPARE_PROMPT` - Test-Prompt für den Vergleich
- `COMPARE_RETRIES` - Wiederholungsversuche für Resilienz (Standard: 2)
- `FOUNDRY_LOCAL_ENDPOINT` - Service-Endpunkt überschreiben (automatisch erkannt, falls nicht gesetzt)

**Funktionsweise (Offizielles SDK-Muster):**
1. **FoundryLocalManager** initialisiert und verwaltet den Foundry Local Service
2. Der Service startet automatisch, falls er nicht läuft (keine manuelle Einrichtung erforderlich)
3. Modelle werden automatisch von Aliasnamen zu konkreten IDs aufgelöst
4. Hardware-optimierte Varianten werden ausgewählt (CUDA, NPU oder CPU)
5. OpenAI-kompatibler Client führt Chat-Abschlüsse durch
6. Metriken werden erfasst: Latenz, Tokens, Ausgabequalität
7. Ergebnisse werden verglichen, um das Geschwindigkeitsverhältnis zu berechnen

Dieser Mikro-Vergleich hilft zu entscheiden, wann die Weiterleitung an ein größeres Modell für Ihren Anwendungsfall gerechtfertigt ist.

**SDK-Referenz:** 
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- Workshop-Utils: Verwendet das offizielle Muster aus ../samples/workshop_utils.py

**Wichtige Vorteile:**
- ✅ Automatische Service-Erkennung und Initialisierung
- ✅ Automatischer Start des Services, falls nicht aktiv
- ✅ Eingebaute Modellauflösung und Caching
- ✅ Hardware-Optimierung (CUDA/NPU/CPU)
- ✅ OpenAI-SDK-Kompatibilität
- ✅ Robuste Fehlerbehandlung mit Wiederholungsversuchen
- ✅ Lokale Inferenz (keine Cloud-API erforderlich)


## 🚨 Voraussetzungen: Foundry Local muss laufen!

**Bevor Sie dieses Notebook ausführen**, stellen Sie sicher, dass der Foundry Local-Dienst eingerichtet ist:

### Schnellstart-Befehle (im Terminal ausführen):

```bash
# 1. Start the Foundry Local service
foundry service start

# 2. Load the default models used in this comparison (CPU-optimized)
foundry model run phi-4-mini
foundry model run qwen2.5-3b

# 3. Verify models are loaded
foundry model ls

# 4. Check service health
foundry service status
```

### Alternative Modelle (falls die Standardmodelle nicht verfügbar sind):

```bash
# Even smaller alternatives (if memory is very limited)
foundry model run phi-3.5-mini
foundry model run qwen2.5-0.5b

# Or update the environment variables in this notebook:
# SLM_ALIAS = 'phi-3.5-mini'
# LLM_ALIAS = 'qwen2.5-1.5b'  # Or qwen2.5-0.5b for minimum memory
```

⚠️ **Wenn Sie diese Schritte überspringen**, wird beim Ausführen der untenstehenden Notebook-Zellen ein `APIConnectionError` angezeigt.


In [29]:
# Install dependencies
!pip install -q foundry-local-sdk openai numpy requests

### Erklärung: Wichtige Importe
Importiert Zeitmanagement-Utilities sowie Foundry Local- und OpenAI-Clients, die verwendet werden, um Modellinformationen abzurufen und Chat-Abschlüsse durchzuführen.


In [30]:
import os, time, json
from foundry_local import FoundryLocalManager
from openai import OpenAI
import sys
sys.path.append('../samples')
from workshop_utils import get_client, chat_once

### Erklärung: Aliase & Prompt-Einrichtung
Definiert umgebungsabhängige Aliase für kleine vs. größere Modelle sowie einen Vergleichs-Prompt. Passen Sie Umgebungsvariablen an, um mit verschiedenen Modellfamilien oder Aufgaben zu experimentieren.


In [31]:
# Default to CPU models for better memory efficiency
SLM = os.getenv('SLM_ALIAS', 'phi-4-mini')  # Auto-selects CPU variant
LLM = os.getenv('LLM_ALIAS', 'qwen2.5-3b')  # Smaller LLM, more memory-friendly
PROMPT = os.getenv('COMPARE_PROMPT', 'List 5 benefits of local AI inference.')
# Endpoint is now managed by FoundryLocalManager - it auto-detects or can be overridden
ENDPOINT = os.getenv('FOUNDRY_LOCAL_ENDPOINT', None)

### 💡 Speicheroptimierte Konfiguration

**Dieses Notebook verwendet standardmäßig speichereffiziente Modelle:**
- `phi-4-mini` → ~4GB RAM (Foundry Local wählt automatisch die CPU-Variante)
- `qwen2.5-3b` → ~3GB RAM (anstatt 7B, das ~7GB+ benötigt)

**Automatische Port-Erkennung:**
- Foundry Local kann unterschiedliche Ports verwenden (häufig 55769 oder 59959)
- Die Diagnosezelle unten erkennt automatisch den richtigen Port
- Keine manuelle Konfiguration erforderlich!

**Falls Sie weniger als 8GB RAM haben, verwenden Sie noch kleinere Modelle:**
```python
SLM = 'phi-3.5-mini'      # ~2GB
LLM = 'qwen2.5-0.5b'      # ~500MB
```


In [32]:
# Display current configuration
print("="*60)
print("CURRENT CONFIGURATION")
print("="*60)
print(f"SLM Model:     {SLM}")
print(f"LLM Model:     {LLM}")
print(f"SDK Pattern:   FoundryLocalManager (official)")
print(f"Endpoint:      {ENDPOINT or 'Auto-detect'}")
print(f"Test Prompt:   {PROMPT[:50]}...")
print(f"Retry Count:   2")
print("="*60)
print("\n💡 Using official Foundry SDK pattern from workshop_utils")
print("   → FoundryLocalManager handles service lifecycle")
print("   → Automatic model resolution and hardware optimization")
print("   → OpenAI-compatible API for inference")

CURRENT CONFIGURATION
SLM Model:     phi-4-mini
LLM Model:     qwen2.5-7b
SDK Pattern:   FoundryLocalManager (official)
Endpoint:      Auto-detect
Test Prompt:   List 5 benefits of local AI inference....
Retry Count:   2

💡 Using official Foundry SDK pattern from workshop_utils
   → FoundryLocalManager handles service lifecycle
   → Automatic model resolution and hardware optimization
   → OpenAI-compatible API for inference


### Erklärung: Ausführungshelfer (Foundry SDK-Muster)
Verwendet das offizielle Foundry Local SDK-Muster, wie in den Workshop-Beispielen dokumentiert:

**Ansatz:**
- **FoundryLocalManager** - Initialisiert und verwaltet den Foundry Local-Dienst
- **Automatische Erkennung** - Erkennt Endpunkte automatisch und verwaltet den Lebenszyklus des Dienstes
- **Modellauflösung** - Wandelt Aliase in vollständige Modell-IDs um (z. B. phi-4-mini → phi-4-mini-instruct-cpu)
- **Hardware-Optimierung** - Wählt die beste Variante für die verfügbare Hardware aus (CUDA, NPU oder CPU)
- **OpenAI-Client** - Konfiguriert mit dem Endpunkt des Managers für OpenAI-kompatiblen API-Zugriff

**Resilienz-Funktionen:**
- Exponentielles Backoff-Wiederholungslogik (konfigurierbar über die Umgebung)
- Automatischer Dienststart, falls nicht aktiv
- Verbindungsüberprüfung nach der Initialisierung
- Fehlerbehandlung mit detaillierter Fehlerberichterstattung
- Modell-Caching, um wiederholte Initialisierungen zu vermeiden

**Ergebnisstruktur:**
- Latenzmessung (Echtzeit)
- Verfolgung der Token-Nutzung (falls verfügbar)
- Beispielausgabe (gekürzt für bessere Lesbarkeit)
- Fehlerdetails bei fehlgeschlagenen Anfragen

Dieses Muster nutzt das Modul workshop_utils, das dem offiziellen SDK-Muster folgt.

**SDK-Referenz:**
- Haupt-Repository: https://github.com/microsoft/Foundry-Local
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- Workshop Utils: ../samples/workshop_utils.py


In [39]:
def setup(alias: str, endpoint: str = None, retries: int = 3):
    """
    Initialize a Foundry Local model connection using official SDK pattern.
    
    This follows the workshop_utils pattern which uses FoundryLocalManager
    to properly initialize the Foundry Local service and resolve models.
    
    Args:
        alias: Model alias (e.g., 'phi-4-mini', 'qwen2.5-3b')
        endpoint: Optional endpoint override (usually auto-detected)
        retries: Number of connection attempts (default: 3)
    
    Returns:
        tuple: (manager, client, model_id, metadata) or (None, None, alias, error_metadata) if failed
    """
    import time
    
    last_err = None
    current_delay = 2  # seconds
    
    for attempt in range(1, retries + 1):
        try:
            print(f"[Init] Connecting to '{alias}' (attempt {attempt}/{retries})...")
            
            # Use the workshop utility which follows the official SDK pattern
            manager, client, model_id = get_client(alias, endpoint=endpoint)
            
            print(f"[OK] Connected to '{alias}' -> {model_id}")
            print(f"     Endpoint: {manager.endpoint}")
            
            return manager, client, model_id, {
                'endpoint': manager.endpoint,
                'resolved': model_id,
                'attempts': attempt,
                'status': 'success'
            }
            
        except Exception as e:
            last_err = e
            error_msg = str(e)
            
            # Provide helpful error messages
            if "Connection error" in error_msg or "connection refused" in error_msg.lower():
                print(f"[ERROR] Cannot connect to Foundry Local service")
                print(f"        → Is the service running? Try: foundry service start")
                print(f"        → Is the model loaded? Try: foundry model run {alias}")
            elif "not found" in error_msg.lower():
                print(f"[ERROR] Model '{alias}' not found in catalog")
                print(f"        → Available models: Run 'foundry model ls' in terminal")
                print(f"        → Download model: Run 'foundry model download {alias}'")
            else:
                print(f"[ERROR] Setup failed: {type(e).__name__}: {error_msg}")
            
            if attempt < retries:
                print(f"[Retry] Waiting {current_delay:.1f}s before retry...")
                time.sleep(current_delay)
                current_delay *= 2  # Exponential backoff
    
    # All retries failed - provide actionable guidance
    print(f"\n❌ Failed to initialize '{alias}' after {retries} attempts")
    print(f"   Last error: {type(last_err).__name__}: {str(last_err)}")
    print(f"\n💡 Troubleshooting steps:")
    print(f"   1. Ensure Foundry Local service is running:")
    print(f"      → foundry service status")
    print(f"      → foundry service start (if not running)")
    print(f"   2. Ensure model is loaded:")
    print(f"      → foundry model run {alias}")
    print(f"   3. Check available models:")
    print(f"      → foundry model ls")
    print(f"   4. Try alternative models if '{alias}' isn't available")
    
    return None, None, alias, {
        'error': f"{type(last_err).__name__}: {str(last_err)}",
        'endpoint': endpoint or 'auto-detect',
        'attempts': retries,
        'status': 'failed'
    }


def run(client, model_id: str, prompt: str, max_tokens: int = 180, temperature: float = 0.5):
    """
    Run inference with the configured model using OpenAI SDK.
    
    Args:
        client: OpenAI client instance (configured for Foundry Local)
        model_id: Model identifier (resolved from alias)
        prompt: Input prompt
        max_tokens: Maximum response tokens (default: 180)
        temperature: Sampling temperature (default: 0.5)
    
    Returns:
        dict: Response with timing, tokens, and content
    """
    import time
    
    start = time.time()
    
    try:
        response = client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature
        )
        
        elapsed = time.time() - start
        
        # Extract response details
        content = response.choices[0].message.content
        
        # Try to extract token usage from multiple possible locations
        usage_info = {}
        if hasattr(response, 'usage') and response.usage:
            usage_info['prompt_tokens'] = getattr(response.usage, 'prompt_tokens', None)
            usage_info['completion_tokens'] = getattr(response.usage, 'completion_tokens', None)
            usage_info['total_tokens'] = getattr(response.usage, 'total_tokens', None)
        
        # Calculate approximate token count if API doesn't provide it
        # Rough estimate: ~4 characters per token for English text
        if not usage_info.get('total_tokens'):
            estimated_prompt_tokens = len(prompt) // 4
            estimated_completion_tokens = len(content) // 4
            estimated_total = estimated_prompt_tokens + estimated_completion_tokens
            usage_info['estimated_tokens'] = estimated_total
            usage_info['estimated_prompt_tokens'] = estimated_prompt_tokens
            usage_info['estimated_completion_tokens'] = estimated_completion_tokens
        
        return {
            'status': 'success',
            'content': content,
            'elapsed_sec': elapsed,
            'tokens': usage_info.get('total_tokens') or usage_info.get('estimated_tokens'),
            'usage': usage_info,
            'model': model_id
        }
        
    except Exception as e:
        elapsed = time.time() - start
        return {
            'status': 'error',
            'error': f"{type(e).__name__}: {str(e)}",
            'elapsed_sec': elapsed,
            'model': model_id
        }


print("✅ Execution helpers defined: setup(), run()")
print("   → Uses workshop_utils for proper SDK integration")
print("   → setup() initializes with FoundryLocalManager")
print("   → run() executes inference via OpenAI-compatible API")
print("   → Token counting: Uses API data or estimates if unavailable")

✅ Execution helpers defined: setup(), run()
   → Uses workshop_utils for proper SDK integration
   → setup() initializes with FoundryLocalManager
   → run() executes inference via OpenAI-compatible API
   → Token counting: Uses API data or estimates if unavailable


### Erklärung: Selbsttest vor dem Start
Führt einen einfachen Verbindungstest mit FoundryLocalManager für beide Modelle durch. Dies überprüft:
- Der Dienst ist zugänglich
- Modelle können initialisiert werden
- Aliase werden zu tatsächlichen Modell-IDs aufgelöst
- Die Verbindung ist stabil, bevor der Vergleich durchgeführt wird

Die setup()-Funktion verwendet das offizielle SDK-Muster aus workshop_utils.


In [34]:
# Simplified diagnostic: Just verify service is accessible
import requests

def check_foundry_service():
    """Quick diagnostic to verify Foundry Local is running."""
    # Try common ports
    endpoints_to_try = [
        "http://localhost:59959",
        "http://127.0.0.1:59959", 
        "http://localhost:55769",
        "http://127.0.0.1:55769",
    ]
    
    print("[Diagnostic] Checking Foundry Local service...")
    
    for endpoint in endpoints_to_try:
        try:
            response = requests.get(f"{endpoint}/health", timeout=2)
            if response.status_code == 200:
                print(f"✅ Service is running at {endpoint}")
                
                # Try to list models
                try:
                    models_response = requests.get(f"{endpoint}/v1/models", timeout=2)
                    if models_response.status_code == 200:
                        models_data = models_response.json()
                        model_count = len(models_data.get('data', []))
                        print(f"✅ Found {model_count} models available")
                        if model_count > 0:
                            print("   Models:", [m.get('id', 'unknown') for m in models_data.get('data', [])[:5]])
                except Exception as e:
                    print(f"⚠️  Could not list models: {e}")
                
                return endpoint
        except requests.exceptions.ConnectionError:
            continue
        except Exception as e:
            print(f"⚠️  Error checking {endpoint}: {e}")
    
    print("\n❌ Foundry Local service not found!")
    print("\n💡 To fix this:")
    print("   1. Open a terminal")
    print("   2. Run: foundry service start")
    print("   3. Run: foundry model run phi-4-mini")
    print("   4. Run: foundry model run qwen2.5-3b")
    print("   5. Re-run this notebook")
    return None

# Run diagnostic
discovered_endpoint = check_foundry_service()

if discovered_endpoint:
    print(f"\n✅ Service detected (will be managed by FoundryLocalManager)")
else:
    print(f"\n⚠️  No service detected - FoundryLocalManager will attempt to start it")

[Diagnostic] Checking Foundry Local service...

❌ Foundry Local service not found!

💡 To fix this:
   1. Open a terminal
   2. Run: foundry service start
   3. Run: foundry model run phi-4-mini
   4. Run: foundry model run qwen2.5-3b
   5. Re-run this notebook

⚠️  No service detected - FoundryLocalManager will attempt to start it


In [35]:
# Quick Fix: Start service and load models from notebook
# Uncomment the commands you need:

# !foundry service start
# !foundry model run phi-4-mini
# !foundry model run qwen2.5-3b
# !foundry model ls

print("⚠️  The commands above are commented out.")
print("Uncomment them if you want to start the service from the notebook.")
print("")
print("💡 Recommended: Run these commands in a separate terminal instead:")
print("   foundry service start")
print("   foundry model run phi-4-mini")
print("   foundry model run qwen2.5-3b")

⚠️  The commands above are commented out.
Uncomment them if you want to start the service from the notebook.

💡 Recommended: Run these commands in a separate terminal instead:
   foundry service start
   foundry model run phi-4-mini
   foundry model run qwen2.5-3b


### 🛠️ Schnelle Lösung: Foundry Local vom Notebook aus starten (Optional)

Falls die obenstehende Diagnose zeigt, dass der Dienst nicht läuft, können Sie versuchen, ihn von hier aus zu starten:

**Hinweis:** Dies funktioniert am besten unter Windows. Auf anderen Plattformen verwenden Sie Terminal-Befehle.


### ⚠️ Fehlerbehebung bei Verbindungsproblemen

Wenn Sie `APIConnectionError` sehen, läuft der Foundry Local-Dienst möglicherweise nicht oder Modelle sind nicht geladen. Versuchen Sie diese Schritte:

**1. Dienststatus überprüfen:**
```bash
# In a terminal (not in notebook):
foundry service status
```

**2. Dienst starten (falls nicht aktiv):**
```bash
foundry service start
```

**3. Erforderliche Modelle laden:**
```bash
# Load the models needed for comparison
foundry model run phi-4-mini
foundry model run qwen2.5-7b

# Or use alternative models:
foundry model run phi-3.5-mini
foundry model run qwen2.5-3b
```

**4. Verfügbarkeit der Modelle überprüfen:**
```bash
foundry model ls
```

**Häufige Probleme:**
- ❌ Dienst läuft nicht → Führen Sie `foundry service start` aus
- ❌ Modelle nicht geladen → Führen Sie `foundry model run <model-name>` aus
- ❌ Portkonflikte → Überprüfen Sie, ob ein anderer Dienst den Port verwendet
- ❌ Firewall blockiert → Stellen Sie sicher, dass lokale Verbindungen erlaubt sind

**Schnelle Lösung:** Führen Sie die Diagnosezelle unten aus, bevor Sie die Vorabprüfung durchführen.


In [36]:
preflight = {}
retries = 2  # Number of retry attempts

for a in (SLM, LLM):
    mgr, c, mid, info = setup(a, endpoint=ENDPOINT, retries=retries)
    # Keep the original status from info (either 'success' or 'failed')
    preflight[a] = info

print('\n[Pre-flight Check]')
for alias, details in preflight.items():
    status_icon = '✅' if details['status'] == 'success' else '❌'
    print(f"  {status_icon} {alias}: {details['status']} - {details.get('resolved', details.get('error', 'unknown'))}")

preflight

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1

[Pre-flight Check]
  ✅ phi-4-mini: success - Phi-4-mini-instruct-cuda-gpu:4
  ✅ qwen2.5-7b: success - qwen2.5-7b-instruct-cuda-gpu:3


{'phi-4-mini': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'Phi-4-mini-instruct-cuda-gpu:4',
  'attempts': 1,
  'status': 'success'},
 'qwen2.5-7b': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'qwen2.5-7b-instruct-cuda-gpu:3',
  'attempts': 1,
  'status': 'success'}}

### ✅ Vorabprüfung: Modellverfügbarkeit

Diese Zelle überprüft, ob beide Modelle über den konfigurierten Endpunkt erreichbar sind, bevor der Vergleich durchgeführt wird.


### Erklärung: Vergleich ausführen & Ergebnisse sammeln
Iteriert über beide Aliase mithilfe des offiziellen Foundry-SDK-Musters:
1. Initialisiere jedes Modell mit setup() (verwendet FoundryLocalManager)
2. Führe Inferenz mit einer OpenAI-kompatiblen API aus
3. Erfasse Latenz, Tokens und Beispielausgabe
4. Erstelle eine JSON-Zusammenfassung mit vergleichender Analyse

Dies folgt demselben Muster wie die Workshop-Beispiele in session04/model_compare.py.


In [40]:
results = []
retries = 2  # Number of retry attempts

for alias in (SLM, LLM):
    mgr, client, mid, info = setup(alias, endpoint=ENDPOINT, retries=retries)
    if client:
        r = run(client, mid, PROMPT)
        results.append({'alias': alias, **r})
    else:
        # If setup failed, record error
        results.append({
            'alias': alias,
            'status': 'error',
            'error': info.get('error', 'Setup failed'),
            'elapsed_sec': 0,
            'tokens': None,
            'model': alias
        })

# Display results
print(json.dumps(results, indent=2))

# Quick comparative view
print('\n' + '='*80)
print('COMPARISON SUMMARY')
print('='*80)
print(f"{'Alias':<20} {'Status':<15} {'Latency(s)':<15} {'Tokens':<15}")
print('-'*80)

for row in results:
    status = row.get('status', 'unknown')
    status_icon = '✅' if status == 'success' else '❌'
    latency_str = f"{row.get('elapsed_sec', 0):.3f}" if row.get('elapsed_sec') else 'N/A'
    
    # Handle token display - show if available or indicate estimated
    tokens = row.get('tokens')
    usage = row.get('usage', {})
    if tokens:
        if 'estimated_tokens' in usage:
            tokens_str = f"~{tokens} (est.)"
        else:
            tokens_str = str(tokens)
    else:
        tokens_str = 'N/A'
    
    print(f"{status_icon} {row['alias']:<18} {status:<15} {latency_str:<15} {tokens_str:<15}")

print('-'*80)

# Show detailed token breakdown if available
print("\nDetailed Token Usage:")
for row in results:
    if row.get('status') == 'success' and row.get('usage'):
        usage = row['usage']
        print(f"\n  {row['alias']}:")
        if 'prompt_tokens' in usage and usage['prompt_tokens']:
            print(f"    Prompt tokens:     {usage['prompt_tokens']}")
            print(f"    Completion tokens: {usage['completion_tokens']}")
            print(f"    Total tokens:      {usage['total_tokens']}")
        elif 'estimated_tokens' in usage:
            print(f"    Estimated prompt:     {usage['estimated_prompt_tokens']}")
            print(f"    Estimated completion: {usage['estimated_completion_tokens']}")
            print(f"    Estimated total:      {usage['estimated_tokens']}")
            print(f"    (API did not provide token counts - using ~4 chars/token estimate)")

print('\n' + '='*80)

# Calculate speedup if both succeeded
if len(results) == 2 and all(r.get('status') == 'success' and r.get('elapsed_sec') for r in results):
    speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec']
    print(f"\n💡 SLM is {speedup:.2f}x faster than LLM for this prompt")
    
    # Compare token throughput if available
    slm_tokens = results[0].get('tokens', 0)
    llm_tokens = results[1].get('tokens', 0)
    if slm_tokens and llm_tokens:
        slm_tps = slm_tokens / results[0]['elapsed_sec']
        llm_tps = llm_tokens / results[1]['elapsed_sec']
        print(f"   SLM throughput: {slm_tps:.1f} tokens/sec")
        print(f"   LLM throughput: {llm_tps:.1f} tokens/sec")
        
elif any(r.get('status') == 'error' for r in results):
    print(f"\n⚠️  Some models failed - check errors above")
    print("   Ensure Foundry Local is running: foundry service start")
    print("   Ensure models are loaded: foundry model run <model-name>")

results

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[
  {
    "alias": "phi-4-mini",
    "status": "success",
    "content": "1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the n

[{'alias': 'phi-4-mini',
  'status': 'success',
  'content': '1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the need for data transmission over the network, which can save bandwidth and reduce the risk of network congestion.\n\n4. Improved Reliability: Local processing can be more reliable, as it is less dependent on network connectivity. This is particularly important in scenarios where network connectivity is unreliable or intermittent.\n\n5. Scalability: Local AI inference can be easily scaled by adding more local processing units, making it easier to handle increasing data volumes or m

### Ergebnisse interpretieren

**Wichtige Kennzahlen:**
- **Latenz**: Niedriger ist besser – zeigt schnellere Antwortzeiten an
- **Tokens**: Höherer Durchsatz = mehr verarbeitete Tokens
- **Route**: Bestätigt, welcher API-Endpunkt verwendet wurde

**Wann SLM vs. LLM verwenden:**
- **SLM (Small Language Model)**: Schnelle Antworten, geringer Ressourcenverbrauch, geeignet für einfache Aufgaben
- **LLM (Large Language Model)**: Höhere Qualität, bessere Argumentation, verwenden, wenn Qualität am wichtigsten ist

**Nächste Schritte:**
1. Probieren Sie verschiedene Prompts aus, um zu sehen, wie die Komplexität den Vergleich beeinflusst
2. Experimentieren Sie mit anderen Modellpaaren
3. Nutzen Sie die Workshop-Router-Beispiele (Session 06), um basierend auf der Aufgabenkomplexität intelligent zu routen


In [38]:
# Final Validation Check
print("="*70)
print("VALIDATION SUMMARY")
print("="*70)
print(f"✅ SLM Model: {SLM}")
print(f"✅ LLM Model: {LLM}")
print(f"✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager")
print(f"✅ Pre-flight passed: {all(v['status'] == 'success' for v in preflight.values()) if 'preflight' in dir() else 'Not run yet'}")
print(f"✅ Comparison completed: {len(results) == 2 if 'results' in dir() else 'Not run yet'}")
print(f"✅ Both models responded: {all(r.get('status') == 'success' for r in results) if 'results' in dir() and results else 'Not run yet'}")
print("="*70)

# Check for common configuration issues
issues = []
if 'LLM' in dir() and LLM not in ['qwen2.5-3b', 'qwen2.5-0.5b', 'qwen2.5-1.5b', 'qwen2.5-7b', 'phi-3.5-mini']:
    issues.append(f"⚠️  LLM is '{LLM}' - expected qwen2.5-3b for memory efficiency")
if 'preflight' in dir() and not all(v['status'] == 'success' for v in preflight.values()):
    issues.append("⚠️  Pre-flight check failed - models not accessible")
if 'results' in dir() and results and not all(r.get('status') == 'success' for r in results):
    issues.append("⚠️  Comparison incomplete - check for errors above")

if not issues and 'results' in dir() and results and all(r.get('status') == 'success' for r in results):
    print("🎉 ALL CHECKS PASSED! Notebook completed successfully.")
    print(f"   SLM ({SLM}) vs LLM ({LLM}) comparison completed.")
    if len(results) == 2:
        speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec'] if results[0]['elapsed_sec'] > 0 else 0
        print(f"   Performance: SLM is {speedup:.2f}x faster")
elif issues:
    print("\n⚠️  Issues detected:")
    for issue in issues:
        print(f"   {issue}")
    print("\n💡 Troubleshooting:")
    print("   1. Ensure service is running: foundry service start")
    print("   2. Load models: foundry model run phi-4-mini && foundry model run qwen2.5-7b")
    print("   3. Check model list: foundry model ls")
else:
    print("\n💡 Run all cells above first, then re-run this validation.")
print("="*70)

VALIDATION SUMMARY
✅ SLM Model: phi-4-mini
✅ LLM Model: qwen2.5-7b
✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager
✅ Pre-flight passed: True
✅ Comparison completed: True
✅ Both models responded: True
🎉 ALL CHECKS PASSED! Notebook completed successfully.
   SLM (phi-4-mini) vs LLM (qwen2.5-7b) comparison completed.
   Performance: SLM is 5.14x faster



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
